<a href="https://colab.research.google.com/github/ashutoshgithubs/Insurance_Chatbot/blob/main/Insurance_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit PyPDF2 langchain huggingface-hub sentence-transformers faiss-cpu

import os
import tempfile
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
import getpass
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate

if 'HUGGINGFACEHUB_API_TOKEN' not in os.environ:
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass("Please enter your HuggingFace API token: ")

# Setting up the Streamlit
st.set_page_config(
    page_title="Insurance Policy AI Assistant",
    page_icon="📄",
    layout="centered"
)

# Adding some CSS for better UI ---
st.markdown("""
<style>
    .stButton>button {
        background-color: #4CAF50;
        color: white;
        border-radius: 5px;
    }
    .stTextInput>div>div>input {
        border: 1px solid #4CAF50;
    }
    .sidebar .sidebar-content {
        background-color: #f0f2f6;
    }
</style>
""", unsafe_allow_html=True)

st.title("📄 Insurance Policy AI Assistant")
st.markdown("""
Upload your **insurance policy PDFs**, and I'll answer any questions you have about coverage, premiums, claims, and more.
*No pre-loaded data – your documents stay private.*
""")

# File Upload Section in form of pdf
uploaded_files = st.file_uploader(
    "**Upload Insurance Policy PDFs**",
    type=["pdf"],
    accept_multiple_files=True
)

# Now let's process the uploaded files
def extract_text_from_pdf(pdf_file):
    text = ""
    pdf_reader = PdfReader(pdf_file)
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

def create_knowledge_base(texts):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(texts)

    # Generating embeddings.....
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    knowledge_base = FAISS.from_texts(chunks, embeddings)

    return knowledge_base

# Let's initialize the Q&A Chain
def get_qa_chain(knowledge_base):
    prompt_template = """
    You are an insurance policy expert. Answer ONLY based on the provided context.

    **Context:**
    {context}

    **Question:**
    {question}

    **Answer clearly and concisely:**
    """

    PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )

    # Using HuggingFace's Flan-T5 model
    llm = HuggingFaceHub(
        repo_id="google/flan-t5-large",
        model_kwargs={"temperature": 0, "max_length": 512}
    )

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=knowledge_base.as_retriever(),
        chain_type_kwargs={"prompt": PROMPT},
        return_source_documents=True
    )

    return qa_chain


if uploaded_files:
    # Extracting the texts from all uploaded PDFs
    combined_text = ""
    for file in uploaded_files:
        combined_text += extract_text_from_pdf(file)

    knowledge_base = create_knowledge_base(combined_text)
    qa_chain = get_qa_chain(knowledge_base)

    st.success("✅ Documents processed! Ask me anything about your policies.")

    # Now creating the Chat Interface
    user_question = st.text_input("**Ask a question about your policy:**", placeholder="e.g., What's covered in my health insurance?")

    if user_question:
        response = qa_chain({"query": user_question})

        st.subheader("Answer:")
        st.write(response["result"])

        st.markdown("**Source:**")
        st.write(response["source_documents"][0].metadata["source"])
else:
    st.warning("⚠️ Please upload at least one insurance policy PDF to get started.")

# Adding Info. on UI to showcase the working of my chat bot.
with st.sidebar:
    st.header("How It Works")
    st.markdown("""
    1. **Upload** your insurance policy PDFs
    2. The AI **reads & processes** the documents
    3. **Ask questions** about coverage, claims, etc.
    4. Get **accurate answers** based on your policies
    """)

    st.divider()
    st.markdown("**Note:** Your documents are **not stored** after the session ends.")